In [116]:
from elasticsearch import Elasticsearch

Definiendo variables globales e instanciando
elasticserach service

In [131]:
HOST = 'localhost:9200'

es = Elasticsearch(hosts=HOST)
es.indices.get("_all")

{'.kibana': {'aliases': {},
  'mappings': {'properties': {'config': {'properties': {'buildNum': {'type': 'long'}}},
    'type': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'updated_at': {'type': 'date'}}},
  'settings': {'index': {'creation_date': '1580752202642',
    'number_of_shards': '1',
    'number_of_replicas': '1',
    'uuid': '4Obko96aQYu7ucjTBmFknA',
    'version': {'created': '7050299'},
    'provided_name': '.kibana'}}}}

Crear un  index llamado **es_workshop** con mapping tipo parent-child, donde:

**Parent**: Usuario
* user_name
* user_code
* user_time_stamp

**Child**: Sensor
* Todas las caracteristicas de las medidas ingresadas


In [137]:
mapping = {
  "settings": {
    "number_of_replicas": 0,
    "number_of_shards": 1
  },
  "mappings": {
    "properties": {
      "user_name":{
        "type": "text"
      },
      "user_code":{
        "type": "keyword"
      },
      "time_stamp":{
        "type": "date",
        "format": "epoch_millis"
      },
      "sensor":{
        "properties": {
          "Timestamp": {
            "type":"date",
            "format":"epoch_millis"
          },
          "Vehicle's speed (in m/s)": {"type": "float"},
          "Shift number (0 = intermediate position)": {"type": "float"},
          "Engine Load (% of max power)": {"type":"float"},
          "Total Acceleration (m/s^2)": {"type": "float"},
          "Engine RPM": {"type": "float"}

        }
      },
      "relation_type":{
        "type": "join",
        "relations":{
          "user":"sensor"
        }
      }
    }
  }
}

In [138]:
index_name = "es_workshop"
resp = es.indices.create(index=index_name, body=mapping)
resp

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es_workshop'}

Indexar usuarios con bulk. Se puede pero no se debe, se pierde la relación **parent**, **child**

In [129]:
# users_json = [
#     {"user_name":"Efra",
#      "user_code": 123,
#      "time_stamp": 1580250575.856432,
#      "relation_type":{"name": "user"}}, 
#     {"user_name":"Lenno",
#      "user_code": 456,
#      "time_stamp": 1000166400.5,
#      "relation_type":{"name": "user"}}]

In [130]:
# from elasticsearch.helpers import bulk
# actions = []
# for i, row in enumerate(users_json):
#     print(row['user_name'], row['user_code'])
#     actions.append({"_op_type": "index",
#                    "_index": index_name,
#                    "_id": i,
#                    "_routing": row['user_name'],
#                    "_source": row})
#     resp = bulk(es, actions=actions)
# resp


Efra 123
Lenno 456


(2, [])

Ingreso de usuarios con PUT. Este comando permite mantener las relaciones de **routing**

In [139]:
user_1={
  "user_name":"Efra",
  "user_code": 123,
  "time_stamp": 1580250575.856432,
  "relation_type":{"name": "user"}
}

resp = es.index(index=index_name, id=1, body=user_1, routing="Efra")
resp

{'_index': 'es_workshop',
 '_type': '_doc',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [140]:
user_2={
  "user_name":"Lenno",
  "user_code": 456,
  "time_stamp": 1000166400.5,
  "relation_type":{"name": "user"}
}
resp = es.index(index=index_name, id=2, body=user_2, routing="Lenno")
resp

{'_index': 'es_workshop',
 '_type': '_doc',
 '_id': '2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

Ingresando datos a cada usuario con **bulk**. El formato presentado permite mandar los datos como bulk, pero los datos no son debidamente ingresados en el es

In [109]:
# data = [
#         {"Timestamp": 1000166400.5,
#          "Vehicle's speed (in m/s)": 0.055034000000000007,
#          "Shift number (0 = intermediate position)": 0.0,
#          "Engine Load (% of max power)": -1.0,
#          "Total Acceleration (m/s^2)": 0.020479,
#          "Engine RPM": 608.63, "relation_type":{"name":"sensor", "parent":1}}, 
#         {"Timestamp": 1000166400.51,
#          "Vehicle's speed (in m/s)": 0.056112999999999996,
#          "Shift number (0 = intermediate position)": 0.0,
#          "Engine Load (% of max power)": -1.0,
#          "Total Acceleration (m/s^2)": 0.007929899999999998,
#          "Engine RPM": 623.47, "relation_type":{"name":"sensor", "parent":1}},
#         {"Timestamp": 1000166400.53,
#          "Vehicle's speed (in m/s)": 0.060233,
#          "Shift number (0 = intermediate position)": 0.0,
#          "Engine Load (% of max power)": -1.0,
#          "Total Acceleration (m/s^2)": -0.009987200000000002,
#          "Engine RPM": 653.14,"relation_type":{ "name": "sensor", "parent": 2}}]


Diccionario padre - nombre

In [110]:
# users_parent_dict = {1:"Efra", 2:"Lenno"}

Ingresando la data en bulk

In [111]:
# actions = []
# for i, row in enumerate(data):
#     user_parent_number = row['relation_type']['parent']
#     actions.append({"_op_type": "index",
#                    "_index": index_name,
#                    "_id": "p"+str(user_parent_number)+"-"+str(i),
#                    "_routing": users_parent_dict[user_parent_number],
#                    "_source": row})
#     resp = bulk(es, actions=actions)
# resp


Ingresando datos de medidas para usuario 1

In [141]:
datax = {"sensor":{
              "Timestamp": 2222166400.5,
              "Vehicle's speed (in m/s)": 2.2552347,
              "Shift number (0 = intermediate position)": 2.2,
              "Engine Load (% of max power)": -2.0,
              "Total Acceleration (m/s^2)": 2.020479,
              "Engine RPM": 2608.63
            
  }, "relation_type":{"name":"sensor", "parent":1}}

resp = es.index(index=index_name, id='u123-1', body=datax, routing="Efra")
resp



{'_index': 'es_workshop',
 '_type': '_doc',
 '_id': 'u123-1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

In [142]:
datax = {"sensor":{
              "Timestamp": 1000166400.51,
              "Vehicle's speed (in m/s)": 0.056112999999999996,
              "Shift number (0 = intermediate position)": 0.0,
              "Engine Load (% of max power)": -1.0,
              "Total Acceleration (m/s^2)": 0.007929899999999998,
              "Engine RPM": 623.47
            
  }, "relation_type":{"name":"sensor", "parent":1}}
resp = es.index(index=index_name, id='u123-2', body=datax, routing="Efra")
resp

{'_index': 'es_workshop',
 '_type': '_doc',
 '_id': 'u123-2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 3,
 '_primary_term': 1}

Ingresando un dato inicial en **user 2**

In [143]:
datax = {
     "sensor":{"Timestamp": 1000166400.53,
     "Vehicle's speed (in m/s)": 0.060233,
       "Shift number (0 = intermediate position)": 0.0,
       "Engine Load (% of max power)": -1.0,
       "Total Acceleration (m/s^2)": -0.009987200000000002,
       "Engine RPM": 653.14},
     "relation_type":{"name": "sensor", "parent": 2}}

resp = es.index(index=index_name, id='u456-1', body=datax, routing="Lenno")
resp

{'_index': 'es_workshop',
 '_type': '_doc',
 '_id': 'u456-1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 4,
 '_primary_term': 1}

In [149]:
datax = {
        "sensor":{"Timestamp": 1111111.11,
        "Vehicle's speed (in m/s)": 6.666233,
        "Shift number (0 = intermediate position)": 1.1,
        "Engine Load (% of max power)": -6.6,
        "Total Acceleration (m/s^2)": -6.0002,
        "Engine RPM": 1200.00},
        "relation_type":{"name": "sensor", "parent": 2}
}
resp = es.index(index=index_name, id='u456-2', body=datax, routing='Lenno')
resp

{'_index': 'es_workshop',
 '_type': '_doc',
 '_id': 'u456-2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 5,
 '_primary_term': 1}

Obtener todos los hijos de un usuario por Código de Usuario

In [150]:
resp = es.search(index=index_name, body={"query": {"has_parent": {"parent_type": "user",
                                                           "query": {"match": {"user_code": "456" }}}}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print("{}: {}".format(hit["_routing"],hit["_source"]))

Got 2 Hits:
Lenno: {'sensor': {'Timestamp': 1000166400.53, "Vehicle's speed (in m/s)": 0.060233, 'Shift number (0 = intermediate position)': 0.0, 'Engine Load (% of max power)': -1.0, 'Total Acceleration (m/s^2)': -0.009987200000000002, 'Engine RPM': 653.14}, 'relation_type': {'name': 'sensor', 'parent': 2}}
Lenno: {'sensor': {'Timestamp': 1111111.11, "Vehicle's speed (in m/s)": 6.666233, 'Shift number (0 = intermediate position)': 1.1, 'Engine Load (% of max power)': -6.6, 'Total Acceleration (m/s^2)': -6.0002, 'Engine RPM': 1200.0}, 'relation_type': {'name': 'sensor', 'parent': 2}}
